Question 7


In [2]:
import cv2
import numpy as np

def zoom_image(img, scale, method='bilinear'):
    
    old_h, old_w = img.shape[:2]
    new_h, new_w = int(old_h * scale), int(old_w * scale)
    
    
    if len(img.shape) == 3:
        zoomed = np.zeros((new_h, new_w, img.shape[2]), dtype=np.uint8)
    else:
        zoomed = np.zeros((new_h, new_w), dtype=np.uint8)

    for i in range(new_h):
        for j in range(new_w):
            # Map back to coordinates in the original image
            # We subtract 0.5 to align pixel centers
            src_x = (j + 0.5) / scale - 0.5
            src_y = (i + 0.5) / scale - 0.5

            if method == 'nearest':
                # Round to the nearest integer
                x0 = int(round(src_x))
                y0 = int(round(src_y))
                # Clamp coordinates to stay within image bounds
                x0 = min(max(x0, 0), old_w - 1)
                y0 = min(max(y0, 0), old_h - 1)
                zoomed[i, j] = img[y0, x0]

            elif method == 'bilinear':
                # Get the four surrounding pixels
                x0 = int(np.floor(src_x))
                x1 = min(x0 + 1, old_w - 1)
                y0 = int(np.floor(src_y))
                y1 = min(y0 + 1, old_h - 1)
                
                # Ensure x0, y0 are not out of bounds
                x0 = max(x0, 0)
                y0 = max(y0, 0)

                # Weights
                dx = src_x - x0
                dy = src_y - y0

                # Interpolate
                pixel = (1 - dx) * (1 - dy) * img[y0, x0] + \
                        dx * (1 - dy) * img[y0, x1] + \
                        (1 - dx) * dy * img[y1, x0] + \
                        dx * dy * img[y1, x1]
                
                zoomed[i, j] = pixel.astype(np.uint8)

    return zoomed

def calculate_ssd(img1, img2):
   
   
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
   
    diff = img1.astype(np.float32) / 255.0 - img2.astype(np.float32) / 255.0
    ssd = np.sum(diff**2) / (img1.shape[0] * img1.shape[1])
    return ssd


original = cv2.imread('im01.png')
small = cv2.imread('im01small.png')

# Scale up factor (calculate based on original size)
scale_factor = original.shape[1] / small.shape[1]

# Apply Zooming
nearest_zoomed = zoom_image(small, scale_factor, method='nearest')
bilinear_zoomed = zoom_image(small, scale_factor, method='bilinear')

# Calculate SSD
ssd_nearest = calculate_ssd(original, nearest_zoomed)
ssd_bilinear = calculate_ssd(original, bilinear_zoomed)

print(f"SSD (Nearest-Neighbor): {ssd_nearest:.6f}")
print(f"SSD (Bilinear): {ssd_bilinear:.6f}")

# Save results
cv2.imwrite('result_nearest.png', nearest_zoomed)
cv2.imwrite('result_bilinear.png', bilinear_zoomed)

SSD (Nearest-Neighbor): 0.006287
SSD (Bilinear): 0.005311


True


im01   4.0×
im02   4.0×
im03   4.0×
taylor 5.0×,